In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
import numpy as np
import pandas as pd
import sys
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import pickle
from collections import OrderedDict
import copy
from scipy.sparse import csr_matrix
from scipy import io
import seaborn as sns
import joblib
# from base import *
from joblib import Parallel, delayed
import random

## TEST NB

In [25]:
MNM_nb_folder = os.path.join('..', '..', '..', 'side_project', 'network_builder')
sys.path.append(MNM_nb_folder)
python_lib_folder = os.path.join('..', '..', 'pylib')
sys.path.append(python_lib_folder)

In [27]:
from MNMAPI import *
from MNM_mcnb import *
from mcDODE import *

In [5]:
origin_spc_folder = os.path.join('/home/lemma/Documents/MAC-POSTS/src/examples/mcDODE',
                                 'input_files_MckeesRocks_SPC')
cache_folder = os.path.join('cahce')

In [6]:
nb = MNM_network_builder()

In [7]:
nb.load_from_folder(origin_spc_folder)

In [8]:
nb.config

[DTA]
network_name = Snap_graph
unit_time = 5
total_interval = 5000
assign_frq = 180
start_assign_interval = 0
max_interval = 24
flow_scalar = 2
num_of_link = 16110
num_of_node = 6297
num_of_O = 283
num_of_D = 283
OD_pair = 80089
routing_type = Fixed
adaptive_ratio = 0.5

[STAT]
rec_mode = LRn
rec_mode_para = 2
rec_folder = record
rec_volume = 0
volume_load_automatic_rec = 0
volume_record_automatic_rec = 0
rec_tt = 1
tt_load_automatic_rec = 0
tt_record_automatic_rec = 0

[FIXED]
path_file_name = path_table
num_path = 216720
choice_portion = Buffer
route_frq = 180

In [11]:
path_len = len(nb.path_table.ID2path) * nb.config.config_dict['DTA']['max_interval']
car_flow = np.random.rand(path_len) / 100
truck_flow = np.zeros(path_len)

In [12]:
np.sum(car_flow)

26009.780534440855

In [13]:
nb.update_demand_path2(car_flow, truck_flow)

In [14]:
nb.dump_to_folder(cache_folder)

## Load data

In [6]:
# nb_link_ID_list = list(map(lambda x: x.ID, nb.link_list))

In [7]:
# pickle.dump(nb_link_ID_list, open('nb_link_ID_list.pickle', 'wb'))

In [17]:
[m_car, L_car, m_truck, L_truck, m_spd_car, m_spd_truck, observed_link_list] = pickle.load(open('mc_data.pickle', 'rb'))

In [18]:
print len(observed_link_list)

2910


## TEST DTA

In [9]:
# ass_freq = nb.config.config_dict['DTA']['assign_frq']
# num_loading_interval = nb.config.config_dict['DTA']['max_interval'] * ass_freq

In [10]:
ass_freq = 180
num_loading_interval = 24 * 180
num_path = 216720

In [11]:
dta = mcdta_api()

In [12]:
dta.initialize(cache_folder)

0L

In [13]:
dta.register_links(observed_link_list)

0L

In [14]:
# dta.register_paths(range(nb.config.config_dict['FIXED']['num_path']))
dta.register_paths(range(num_path))

0L

In [15]:
dta.install_cc()
dta.install_cc_tree()

0L

In [16]:
dta.run_whole()

0L

In [17]:
dta.get_car_link_tt(np.array([50, 100]))

array([[0.        , 1.056     ],
       [1.        , 1.344     ],
       [1.17818182, 1.17818182],
       ...,
       [1.70181818, 1.70181818],
       [1.57090909, 1.57090909],
       [0.96      , 0.96      ]])

In [19]:
tt = dta.get_car_dar_matrix(np.arange(0, num_loading_interval, ass_freq), 
                np.arange(0, num_loading_interval, ass_freq) + ass_freq)

In [21]:
tt

array([[ 1.328910e+05, -1.000000e+00,  1.638500e+04,  0.000000e+00,
         5.000000e-01],
       [ 1.599460e+05, -1.000000e+00,  1.638500e+04,  0.000000e+00,
         5.000000e-01],
       [ 1.023680e+05, -1.000000e+00,  1.638500e+04,  0.000000e+00,
         5.000000e-01],
       ...,
       [ 8.319300e+04, -1.000000e+00,  1.007613e+06,  4.140000e+03,
         5.000000e-01],
       [ 8.096300e+04, -1.000000e+00,  1.007613e+06,  4.140000e+03,
         5.000000e-01],
       [ 8.253500e+04, -1.000000e+00,  1.007613e+06,  4.140000e+03,
         5.000000e-01]])

## Test MCDODE lib

In [49]:
from mcDODE import *

In [50]:
[m_car, L_car, m_truck, L_truck, m_spd_car, m_spd_truck, observed_link_list] = pickle.load(open('mc_data.pickle', 'rb'))

In [56]:
data_dict = dict()
data_dict['car_count_agg_L_list'] = [L_car]
data_dict['truck_count_agg_L_list'] = [L_truck]
data_dict['car_link_flow'] = [m_car]
data_dict['truck_link_flow'] = [m_truck]
data_dict['car_link_tt'] = [m_spd_car]
data_dict['truck_link_tt'] = [m_spd_truck]

In [57]:
config = dict()
config['use_car_link_flow'] = True
config['use_car_link_tt'] = False
config['use_truck_link_flow'] = True
config['use_truck_link_tt'] = False
config['car_count_agg'] = True
config['truck_count_agg'] = True
config['link_flow_weight'] = 1
config['link_tt_weight'] = 1
config['num_data'] = 1
config['observed_links'] = observed_link_list
config['paths_list'] = range(nb.config.config_dict['FIXED']['num_path'])

In [58]:
dode = MCDODE(nb, config)

In [59]:
dode.add_data(data_dict)

In [61]:
(car_flow, truck_flow) = dode.init_path_flow(car_scale = 0.01, truck_scale = 0.001)

In [62]:
dta = dode._run_simulation(car_flow, truck_flow)